In [67]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import pywt
import ast 

In [2]:
folder_path = "C:/Users/mirwa/Downloads/partition1_instances/partition1_instances/partition1"
fl = glob.glob(f"{folder_path}/FL/*.csv")
fl_df = [pd.read_csv(file, sep='\t', usecols=range(4)) for file in fl]
df_fl = pd.concat(fl_df, ignore_index=True)

In [3]:
nf_df = pd.read_csv('nf_file.csv', sep = '\t')

In [7]:
df_fl['class'] = 1
nf_df['class'] = 0

In [10]:
nf_df.head()
df_fl.head()

,Timestamp,TOTUSJH,TOTBSQ,TOTPOT,class
0,2011-02-16 23:00:00,1315.120642,1.842497e+10,3.016901e+23,1
1,2011-02-16 23:12:00,1295.291384,1.843622e+10,3.024088e+23,1
2,2011-02-16 23:24:00,1291.072348,1.843172e+10,3.019023e+23,1
3,2011-02-16 23:36:00,1276.641067,1.844119e+10,2.997206e+23,1
4,2011-02-16 23:48:00,1290.970251,1.841171e+10,2.985437e+23,1


In [32]:
df_fl.isna().sum()
nf_df.isna().sum()

TOTUSJH    8453
TOTBSQ     8453
TOTPOT     8453
class         0
dtype: int64

In [31]:
#df_fl['Timestamp'] = pd.to_datetime(df_fl['Timestamp'])
#df_fl.set_index('Timestamp', inplace=True)

#nf_df['Timestamp'] = pd.to_datetime(nf_df['Timestamp'])
#nf_df.set_index('Timestamp', inplace=True)


In [29]:
na_col= df_fl.columns.tolist()
na_col.remove('class')
df_fl[na_col] = df_fl[na_col].fillna(df_fl[na_col].mean())
df_fl.isnull().sum()

TOTUSJH    0
TOTBSQ     0
TOTPOT     0
class      0
dtype: int64

In [33]:
na_col= nf_df.columns.tolist()
na_col.remove('class')
nf_df[na_col] = nf_df[na_col].fillna(nf_df[na_col].mean())
nf_df.isnull().sum()

TOTUSJH    0
TOTBSQ     0
TOTPOT     0
class      0
dtype: int64

In [34]:
num_rows_df1 = len(df_fl)
num_rows_df2_required = num_rows_df1 * 10

# Take a subset of rows from df2 to achieve the desired ratio
df2_subset = nf_df.head(num_rows_df2_required)

# Concatenate df1 and the subset of df2
df_combined = pd.concat([df_fl, df2_subset], ignore_index=True)

In [38]:
class_counts = df_combined['class'].value_counts()
print(class_counts)


0    752400
1     75240
Name: class, dtype: int64


In [76]:
df_combined

,TOTUSJH,TOTBSQ,TOTPOT,class
0,1315.120642,1.842497e+10,3.016901e+23,1
1,1295.291384,1.843622e+10,3.024088e+23,1
2,1291.072348,1.843172e+10,3.019023e+23,1
3,1276.641067,1.844119e+10,2.997206e+23,1
4,1290.970251,1.841171e+10,2.985437e+23,1
...,...,...,...,...
827635,60.093455,5.355921e+08,3.775364e+21,0
827636,57.643425,5.255820e+08,3.573900e+21,0
827637,60.052625,5.295436e+08,3.811322e+21,0
827638,59.215174,5.277095e+08,3.860493e+21,0


In [81]:
df_combined

,TOTUSJH,TOTBSQ,TOTPOT,class
0,1315.120642,1.842497e+10,3.016901e+23,1
1,1295.291384,1.843622e+10,3.024088e+23,1
2,1291.072348,1.843172e+10,3.019023e+23,1
3,1276.641067,1.844119e+10,2.997206e+23,1
4,1290.970251,1.841171e+10,2.985437e+23,1
...,...,...,...,...
827635,60.093455,5.355921e+08,3.775364e+21,0
827636,57.643425,5.255820e+08,3.573900e+21,0
827637,60.052625,5.295436e+08,3.811322e+21,0
827638,59.215174,5.277095e+08,3.860493e+21,0


In [90]:

class_labels = df_combined['class'].unique()
transformed_instances = []
wavelet_level = 5

for class_label in class_labels:
    class_data_totusjh = df_combined[df_combined['class'] == class_label]['TOTUSJH'].values
    class_data_totbsq = df_combined[df_combined['class'] == class_label]['TOTBSQ'].values
    class_data_totpot = df_combined[df_combined['class'] == class_label]['TOTPOT'].values
    
    coeffs_totusjh = pywt.wavedec(class_data_totusjh, 'db4', level=wavelet_level)
    coeffs_totbsq = pywt.wavedec(class_data_totbsq, 'db4', level=wavelet_level)
    coeffs_totpot = pywt.wavedec(class_data_totpot, 'db4', level=wavelet_level)
    
    cA_totusjh, cD5_totusjh, cD4_totusjh, cD3_totusjh, cD2_totusjh, cD1_totusjh = coeffs_totusjh
    cA_totbsq, cD5_totbsq, cD4_totbsq, cD3_totbsq, cD2_totbsq, cD1_totbsq = coeffs_totbsq
    cA_totpot, cD5_totpot, cD4_totpot, cD3_totpot, cD2_totpot, cD1_totpot = coeffs_totpot
    
    transformed_instances.extend(list(zip(
        [class_label] * len(cA_totusjh),
        cA_totusjh, cD5_totusjh, cD4_totusjh, cD3_totusjh, cD2_totusjh, cD1_totusjh,
        cA_totbsq, cD5_totbsq, cD4_totbsq, cD3_totbsq, cD2_totbsq, cD1_totbsq,
        cA_totpot, cD5_totpot, cD4_totpot, cD3_totpot, cD2_totpot, cD1_totpot
    )))



In [91]:
columns = ['class_label',
           'cA_totusjh', 'cD5_totusjh', 'cD4_totusjh', 'cD3_totusjh', 'cD2_totusjh', 'cD1_totusjh',
           'cA_totbsq', 'cD5_totbsq', 'cD4_totbsq', 'cD3_totbsq', 'cD2_totbsq', 'cD1_totbsq',
           'cA_totpot', 'cD5_totpot', 'cD4_totpot', 'cD3_totpot', 'cD2_totpot', 'cD1_totpot']

transformed_df = pd.DataFrame(transformed_instances, columns=columns)
transformed_df

,class_label,cA_totusjh,cD5_totusjh,cD4_totusjh,cD3_totusjh,cD2_totusjh,cD1_totusjh,cA_totbsq,cD5_totbsq,cD4_totbsq,cD3_totbsq,cD2_totbsq,cD1_totbsq,cA_totpot,cD5_totpot,cD4_totpot,cD3_totpot,cD2_totpot,cD1_totpot
0,1,7316.958112,0.990588,-0.048636,-9.573908,7.508336,0.460785,1.042576e+11,-3.273609e+05,2.188881e+05,-2.361528e+06,-3.693860e+06,-7.780114e+05,1.699864e+24,8.007732e+19,1.589878e+20,-1.750949e+21,-2.691602e+20,-5.024201e+19
1,1,7318.239603,3.514795,1.634354,-30.705371,22.285954,4.770899,1.042544e+11,-1.768825e+06,6.030458e+05,-7.143650e+06,-1.220831e+07,-6.054475e+06,1.700667e+24,4.245921e+20,8.620428e+20,-5.450070e+21,-1.130511e+21,1.880603e+20
2,1,7313.918425,-10.226335,14.620415,14.229458,-11.880552,9.146528,1.042515e+11,-3.219214e+07,-4.587519e+06,1.198016e+06,7.171198e+06,-1.944961e+07,1.699724e+24,1.960073e+21,3.929860e+21,3.364713e+21,-6.773097e+20,2.668885e+20
3,1,7313.058873,-154.743699,33.682014,23.569551,-20.745290,-10.606851,1.042628e+11,3.814032e+07,4.199503e+07,6.436931e+06,4.671443e+07,-2.434469e+07,1.700130e+24,1.576592e+21,3.754195e+21,1.421511e+21,1.502143e+21,1.412442e+20
4,1,7341.078475,-155.367626,-86.666085,-17.626464,-1.969862,13.535005,1.041348e+11,-1.849596e+09,-1.122839e+08,-5.183105e+07,8.748656e+06,9.576298e+06,1.700729e+24,-1.196768e+22,-2.049312e+21,-3.200107e+21,5.352154e+20,7.937079e+20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25872,0,339.569032,12.612464,-100.996863,-170.423015,3.663383,-5.100722,2.725297e+09,1.076957e+08,-1.343914e+08,-2.515172e+09,2.448622e+07,-1.224931e+06,1.955220e+22,7.987357e+20,-7.494961e+21,-1.785227e+22,5.631009e+19,9.289705e+19
25873,0,344.839647,-3.168161,-206.608496,-140.251328,-3.174293,-1.004700,2.925665e+09,-1.046598e+08,-1.264903e+09,-8.059376e+08,-1.160690e+07,2.411610e+05,2.092696e+22,-1.568980e+20,-6.194574e+21,-4.955172e+21,3.044731e+20,4.795455e+19
25874,0,347.116234,0.093883,350.543472,19.476376,2.818053,1.456797,2.816015e+09,1.032703e+08,1.721694e+09,-1.687478e+08,3.986103e+07,6.242020e+06,1.983772e+22,-6.894403e+19,1.438008e+22,-3.961256e+21,1.867224e+20,-8.807026e+19
25875,0,348.811580,-6.613178,-266.093812,36.149132,-7.603208,-0.949755,3.006107e+09,-6.440291e+07,-1.316300e+09,2.524830e+08,-1.033855e+07,-1.726701e+05,2.181671e+22,-4.110240e+20,-1.805364e+22,5.252662e+21,3.289070e+20,-2.079879e+19


In [73]:
def apply_wavelet_transform(feature_array):
    coefficients = pywt.wavedec(feature_array, 'db4', level=4)
    flattened_coefficients = [item for sublist in coefficients for item in sublist]
    return flattened_coefficients

columns_to_transform = ['TOTUSJH', 'TOTBSQ', 'TOTPOT']

In [74]:
#for feature_column in features.columns:
 #   features[feature_column + '_wavelet'] = features[feature_column].apply(apply_wavelet_transform)


for feature_column in columns_to_transform:
    df_combined[feature_column + '_wavelet'] = df_combined[feature_column].apply(apply_wavelet_transform)


AxisError: Axis greater than data dimensions